In [1]:
# !pip uninstall httpx httpcore
# !pip install --upgrade httpx httpcore

In [1]:
import pandas as pd
import os
from openai import OpenAI
import math
import json
import re
import builtins
import csv

In [2]:
with open('config.json') as config_file:
    config = json.load(config_file)
    open_api_key = config['open_api_key']
language = 'Dutch'
client = OpenAI(api_key=open_api_key)

In [3]:
def get_sets_to_do(post_directory):
    os.makedirs(os.path.join(language, 'Vocabulary', post_directory), exist_ok = True)
    donesets = os.listdir(os.path.join(language, 'Vocabulary', post_directory))
    print('Done:', donesets)
    if '.DS_Store' in donesets:
        donesets.remove('.DS_Store')
    allsets = os.listdir(os.path.join(language, 'Vocabulary', 'split_sets'))
    notdonesets = [i for i in allsets if not i in donesets]
    todosets = notdonesets#['900-1000.csv']
    if '.DS_Store' in donesets:
        todosets.remove('.DS_Store')
    print('To Do:', todosets)
    return todosets
# todosets = get_sets_to_do('Split_Sets','ChatGPT_Sets')

In [4]:
frequencydf = pd.read_csv(os.path.join('Frequency Lists', language, 'Clean.csv'))

Clean

VOCABULARY SPLIT

In [5]:
import os
import math

def split_into_studysets(frequencydf, language, initial_splitsize=100, new_splitsize=250):
    os.makedirs(os.path.join(language, 'Vocabulary', 'Split_Sets'), exist_ok=True)
    columns = [language]
    if 'POS' in frequencydf.columns:
        columns.append('POS')
    if 'Translation' in frequencydf.columns:
        columns.append('Translation')
    if 'English' in frequencydf.columns:
        frequencydf.rename(columns={'English': 'Translation'}, inplace=True)
        columns.append('Translation')
    if 'lemma' in frequencydf.columns:
        columns.append('show')
        columns.append('lemma')
    print('columns:', columns)
    
    total_rows = len(frequencydf)
    i = 0
    splitsize = initial_splitsize
    
    while i * splitsize < total_rows:
        if i * splitsize >= 1000:
            splitsize = new_splitsize
            start_index = 1000 + (i - 10) * new_splitsize  # Adjust start index for new splitsize
        else:
            start_index = i * splitsize
        
        end_index = min(start_index + splitsize, total_rows)
        subdf = frequencydf[start_index:end_index][columns]
        subdf.to_csv(language + '/Vocabulary/Split_Sets/' + str(start_index) + '-' + str(end_index) + '.csv')
        
        i += 1
    
    remainder = total_rows % splitsize
    print('remainder=', remainder)
    if remainder > 0:
        remaindersubdf = frequencydf[-remainder:][columns]
        remaindersubdf.to_csv(language + '/Vocabulary/Split_Sets/' + 'Remainder.csv')

    return subdf

# Example usage
df = split_into_studysets(frequencydf, language=language)


columns: ['Dutch', 'POS']
remainder= 0


If you only want to do a subset of files for time purposes

In [6]:
selected_files = ['800-900.csv', '600-700.csv']#, '700-800.csv', '900-1000.csv', '500-600.csv']

def move_unselected_files(language, selected_files):
    # Create directory for spare sets
    spare_split_sets_dir = os.path.join(language, 'Vocabulary', 'Spare_Split_Sets')
    os.makedirs(spare_split_sets_dir, exist_ok=True)

    split_sets_dir = os.path.join(language, 'Vocabulary', 'Split_Sets')
    all_files = os.listdir(split_sets_dir)
    selected_files_set = set(selected_files)

    # Move files not in selected_files to the spare sets directory
    for file_name in all_files:
        if file_name not in selected_files_set:
            os.rename(
                os.path.join(split_sets_dir, file_name),
                os.path.join(spare_split_sets_dir, file_name)
            )
  # example of selected files
move_unselected_files(language, selected_files)

CHATGPT

In [7]:
CHATGPT_todosets = get_sets_to_do('ChatGPT_Sets')

Done: ['.DS_Store', '800-900.csv', '100-200.csv', '600-700.csv', '700-800.csv', '0-100.csv', '300-400.csv', '900-1000.csv', '400-500.csv', '500-600.csv', '200-300.csv']
To Do: []


In [66]:
def generate_sentence_in_target_language(variable_word, POS, words_to_include, model = 'gpt-4o-2024-05-13', tenses = ['present', 'future', 'past']):
    """
    Generate a sentence in Russian using a specific word and part of speech, with words from a limited vocabulary, along with its English translation.

    Parameters:
    variable_word (str): The word to include in the sentence.
    POS (str): The part of speech the word should operate as.
    words_to_include (tuple): A list of words to try to include, and how many
    model (str): The model to use for generating the sentence (default is 'gpt-4o-2024-05-13').
    
    # vocabulary (list): A list of words to use in the sentence.
    # tenses (list)

    Returns:
    str: A sentence in Russian and its English translation, separated by a newline.
    """
    
    print(variable_word, POS)
    words_to_include = list(words_to_include[0].sample(words_to_include[1], replace=False))
    print(words_to_include)
    # vocabulary = vocabulary
    # vocabulary_str = ', '.join(vocabulary)
    prompt = (
              # f"Use only the following Russian words: {vocabulary_str}. "
        
              f"Create a simple sentence in f{language} using basic vocabulary and containing the word '{variable_word}' "
              f"operating as a {POS} part of speech. Also, provide the English translation of the sentence separated by a newline."
              f"Try to include the following words in the sentence: {words_to_include}"
              # f"Use only the following tenses: {tenses}. "
              )

    try:
        # Make the API request
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=50,
            temperature=0.2,
        )

        # Extract and return the output
        output_message = response.choices[0].message.content.strip()
        return output_message

    except Exception as e:
        return f"An error occurred: {str(e)}"

    # Extract and return the output
    output_message = response.choices[0].message.content.strip()
    return output_message

In [54]:
#TEST THE API
for index, row in frequencydf.iloc[304:308].iterrows():
    sentence = generate_sentence_in_target_language(row[language], row.POS, (frequencydf.iloc[300:400][language], 2))
    print(sentence)

hoop noun
['ligt', 'open']
De hoop ligt open voor iedereen.  
The hope is open for everyone.
wou verb
['verder', 'blijft']
Ik wou verder gaan, maar hij blijft hier.
I wanted to go further, but he stays here.
vermoord verb
['werken', 'alsof']
Hij werkt alsof hij vermoord is.

He works as if he has been murdered.
elke pronoun
['pijn', 'gehad']
Elke keer heb ik pijn gehad.
Every time I have had pain.


In [67]:
def chatonset(s_set_df, set_name, n = 2):
    words_to_include = (s_set_df[language], n)
    s_set_df['ChatGPT_Sentence'] = s_set_df.apply(lambda row: generate_sentence_in_target_language(row[language], row['POS'], words_to_include), axis = 1)
    s_set_df.to_csv(os.path.join(language, 'Vocabulary','ChatGPT_Sets', set_name))
    return s_set_df

In [68]:
def do_all_sets(post_directory, func, language = language):
    os.makedirs(os.path.join(language, 'Vocabulary', post_directory), exist_ok = True)
    allsets = os.listdir(os.path.join(language, 'Vocabulary', 'split_sets'))
    todosets = get_sets_to_do(post_directory)
    for s_set in todosets:
        if s_set[0] != '.':
        # # if s_set == '500-600.csv':
        #     # print(s_set)
            s_set_df = pd.read_csv(
                os.path.join(language, 'Vocabulary', 'split_sets', s_set)
            )
            print(s_set_df)
            print(s_set)
            func(s_set_df, s_set)
    return

In [69]:
do_all_sets('ChatGPT_Sets', chatonset)

Done: []
To Do: ['800-900.csv', '600-700.csv']
    Unnamed: 0      Dutch        POS
0          800      geest       noun
1          801     gingen       verb
2          802     straat       noun
3          803    slechts     adverb
4          804  verkeerde  adjective
..         ...        ...        ...
95         895    schelen       verb
96         896      kaart       noun
97         897        rij       noun
98         898        bal       noun
99         899     werkte       verb

[100 rows x 3 columns]
800-900.csv
geest noun
['wonen', 'stomme']
gingen verb
['geraakt', 'bekend']
straat noun
['bestaat', 'zak']
slechts adverb
['begonnen', 'meeste']
verkeerde adjective
['wegwezen', 'nadat']
slaan verb
['bekend', 'geboren']
FBI FBI
['feest', 'liefje']
afgelopen adjective
['zolang', 'schelen']
hotel noun
['keuze', 'feest']
bekend adjective
['raad', 'stom']
ene one
['koud', 'mocht']
ruimte noun
['geraakt', 'www']
lange adjective
['bal', 'totdat']
trouwens adverb
['betaald', 'stomme']
f

ADD TRANSLATION INFORMATION:

In [70]:
from deep_translator import GoogleTranslator

In [77]:
def add_set_translation_information(s_set_df, set_name, code = 'nl'):
    # Apply the translation
    s_set_df['Translation'] = s_set_df.apply(
        lambda row: GoogleTranslator(source= code, target='en').translate(row[language]), axis=1
    )

    # Save to CSV
    s_set_df.to_csv(os.path.join(language, 'Vocabulary/Translated Sets', set_name), index=False)
    return s_set_df

In [78]:
translation_todosets = get_sets_to_do('Translated Sets')

Done: []
To Do: ['800-900.csv', '600-700.csv']


In [79]:
for set in translation_todosets:
    print(set)
    do_all_sets('Translated Sets', add_set_translation_information)

800-900.csv
Done: []
To Do: ['800-900.csv', '600-700.csv']
    Unnamed: 0      Dutch        POS
0          800      geest       noun
1          801     gingen       verb
2          802     straat       noun
3          803    slechts     adverb
4          804  verkeerde  adjective
..         ...        ...        ...
95         895    schelen       verb
96         896      kaart       noun
97         897        rij       noun
98         898        bal       noun
99         899     werkte       verb

[100 rows x 3 columns]
800-900.csv
    Unnamed: 0      Dutch        POS
0          600      nieuw  adjective
1          601        elk    pronoun
2          602       erin      in it
3          603  ontmoeten       verb
4          604    vechten       verb
..         ...        ...        ...
95         695       trek       noun
96         696       meen       verb
97         697     enkele     adverb
98         698   getrouwd  adjective
99         699   gevraagd  adjective

[100 rows x 3 co

Make into Quizlet

In [150]:
def make_quizlet_set(set_name):
    df = pd.read_csv(os.path.join(language, 'Vocabulary', 'ChatGPT_Sets', set_name))
    transdf = pd.read_csv(os.path.join(language, 'Vocabulary', 'Translated Sets', set_name))

    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].apply(lambda x: re.sub(r'\n+', '\n', x))

    # Removing text within parentheses that start with 'see'
    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.replace(r'\(see[^)]*\)', '', regex=True)
    # Removing any extra spaces that might result from the replacement
    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.strip()
    df['quizlet'] = transdf['POS'] + ' : ' + '*' + transdf['Translation'] + '*' + '\n' + df['ChatGPT_Sentence']

    df = df[[language, 'quizlet']]
    os.makedirs(os.path.join(language, 'Vocabulary', 'Quizlet Sets'), exist_ok=True)
    output_path = os.path.join(language, 'Vocabulary', 'Quizlet Sets', set_name.replace('.csv', '.txt'))

    # Saving the data to a tab-separated text file with 3 lines between each row
    with open(output_path, 'w', encoding='utf-8') as outfile:
        for index, row in df.iterrows():
            outfile.write(f"{row[language]}\t{row['quizlet']}\n\n\n")

    print(f"Text file saved to {output_path}")
    return df

In [151]:
#test make_quizlet_set('800-900.csv')

In [156]:
builtin_set = builtins.set
a = builtin_set(os.listdir(os.path.join(language, 'Vocabulary', 'ChatGPT_Sets')))
b = builtin_set(os.listdir(os.path.join(language, 'Vocabulary', 'Translated Sets')))
quizlet_todosets = a.intersection(b)
quizlet_todosets

{'600-700.csv', '800-900.csv'}

In [157]:
for s_set in quizlet_todosets:
    make_quizlet_set(s_set)

Text file saved to Dutch/Vocabulary/Quizlet Sets/800-900.txt
Text file saved to Dutch/Vocabulary/Quizlet Sets/600-700.txt


In [ ]:
# def makequizlet_step2(input_file):
#     output_file = input_file 
#     with open(input_file, 'r', newline='', encoding='utf-8') as infile:
#         reader = csv.reader(infile)
#         rows = list(reader)
# 
#     # Write the output CSV file with extra newlines
#     with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
#         writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)
#         for row in rows:
#             writer.writerow(row)
#             writer.writerow([])  # Add a blank row
#             writer.writerow([])  # Add a blank row
#             writer.writerow([])
# 
#     print(f"Modified CSV file saved as {output_file}")
#     
# for file in os.listdir(os.path.join(language, 'Vocabulary/Quizlet Sets')):
#     if file[0] != '.':
#         print(file)
#         makequizlet_step2(os.path.join(language, 'Vocabulary/Quizlet Sets/' + file))

In [139]:
# import csv
# def make_quizlet_set(set_name):
#     df = pd.read_csv(os.path.join(language, 'Vocabulary', 'ChatGPT_Sets', set_name))
#     transdf = pd.read_csv(os.path.join(language, 'Vocabulary', 'Translated Sets', set_name))
#     
#     df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].apply(lambda x: re.sub(r'\n+', '\n', x))
#     
#     # Removing text within parentheses that start with 'see'
#     df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.replace(r'\(see[^)]*\)', '', regex=True)   
#     # Removing any extra spaces that might result from the replacement
#     df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.strip()
#     df['quizlet'] = transdf['POS'] + ' : ' + '*' + transdf['Translation'] + '*' + '\n\n' + df['ChatGPT_Sentence']
# 
# 
#     df = df[[language, 'quizlet']]
#     os.makedirs(os.path.join(language, 'Vocabulary', 'Quizlet Sets'), exist_ok = True)
#     output_path = os.path.join(language, 'Vocabulary', 'Quizlet Sets', set_name)
#     df.to_csv(output_path, index=False, quoting=csv.QUOTE_MINIMAL)
#     # df.to_csv(output_path, index=False, quoting=csv.QUOTE_NONE, escapechar = ddd)
#     print(f"CSV file saved to {output_path}")
#     return df

In [143]:
# def remove_quotes_from_csv(file_path):
#     # Read the CSV file and remove quotes
#     with open(file_path, 'r', newline='') as infile:
#         reader = csv.reader(infile)
#         rows = [[cell.replace('"', '') for cell in row] for row in reader]
#     
#     # Write the cleaned data back to the CSV file
#     with open(file_path, 'w', newline='') as outfile:
#         writer = csv.writer(outfile)
#         writer.writerows(rows)

In [144]:
# for file in os.listdir('Russian/Vocabulary/Quizlet Sets'):
#     if file[0] != '.':
#         print(file)
#         makequizlet_step2('Russian/Vocabulary/Quizlet Sets/' + file)